<a href="https://colab.research.google.com/github/axmedddddddd/PhyloTreeFinder/blob/main/PhyloTreeFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Dependenices**

In [ ]:
!pip install binarytree
!pip install tensorflow
!pip install keras
!pip install keras-rl2
!pip install gym

**All the necessary imports**

In [ ]:
import binarytree
from binarytree import get_parent
from binarytree import Node
from binarytree import build2

import gym

import numpy as np
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

**Constants**

In [ ]:
root = Node(0)                                                                  # we start with unmutated sequence
global seq_length                                                               # the length of genetic sequences
global num_of_seq                                                               # number of sequences within the tree
global tree_size
seq_length = 10
num_of_seq = 10
tree_size = 75

zero_tree = [0,
            0, 0,
            0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0,
            None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0,
            None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0,
            None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0,
            None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0,
            None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0, None, 0]

**SEQUANCES**

In [ ]:
# Two-dimensional array of the given sequances
Initial_Matrix = np.array([[1, 1, 0, 1, 1, 1, 1, 0, 1, 1],
                  [1, 1, 0, 0, 0, 1, 1, 0, 0, 0],
                  [1, 0, 1, 1, 1, 1, 0, 1, 1, 1],
                  [0, 0, 1, 0, 1, 0, 1, 1, 0, 1],
                  [1, 0, 1, 1, 0, 0, 1, 1, 0, 1],
                  [0, 0, 0, 0, 0, 0, 0 ,0, 0, 1],
                  [0, 1, 1, 0, 0, 0, 1, 0, 1, 1],
                  [1, 1, 0, 0, 1, 1, 1, 0, 1, 1],
                  [0, 0, 1, 1, 1, 1, 1, 0, 0, 1],
                  [0, 0, 0, 0, 1, 0, 0, 1, 0, 1]], np.int)

**Creating a Custom Enviroment with Gym**

In [ ]:
class Phylogenetic_Tree(gym.Env):

  def __init__(self):
    self.action_space = gym.spaces.Box(np.array([0, 0]), np.array([tree_size, seq_length+1]), dtype=np.int)
    self.observation_space = gym.spaces.Box(low=0, high=1, shape=(2, 10, 10), dtype=np.int)

    self.tree = zero_tree
    self.counter = 0
    root = build2(zero_tree)

  def step(self, action):
    self.counter += 1

    action = int(action)
    mutation = action//tree_size
    site = action - (mutation*tree_size)

    reward = 0                                                                  # encouragement and done-flag of an algorithm
    done = False

    if zero_tree[site] != None:
      self.tree[site] = mutation
      root = build2(self.tree)

      if action != 0:
        reward -= 1                                                             # fines for adding mutations
    else:
      root = build2(self.tree)

    M = (root.height + 1)                                                       # width of branch matrix
    N = root.leaf_count                                                         # length of branch matrix

    Branch_Matrix = np.zeros(shape=(seq_length, num_of_seq), dtype=np.int)      # creating an empty branch matrix
    leaves = root.leaves                                                        # writing leaves of our tree into new array

    # creating the branch matrix with sites
    for j in range(N):
      item = leaves[j]
      for i in reversed(range(M-1)):
          Branch_Matrix[i][j] = item[0].values[0]                               # writing the branch values into a branch matrix
          item = get_parent(root, item)                                         # moving from leaves to parent

    # converting array of sites to sequences
    for j in range(N):
      zero_branch = np.zeros(shape = seq_length, dtype=np.int)
      for i in range(M):
        if Branch_Matrix[i][j] != 0:                                            # ignoring the non-mutation nodes
          zero_branch[Branch_Matrix[i][j]-1] = 1                                # changing sites to nucleotides
      for i in range(M):
        Branch_Matrix[i][j] = zero_branch[i]

    Branch_Matrix.transpose                                                     # branches should be listed from top to bottom

    state = np.stack((Branch_Matrix, Initial_Matrix))                           # creating a state vector

    for i in range (num_of_seq):
      for j in range(seq_length):
        if (Branch_Matrix[i][j] == Initial_Matrix[i][j]):
          if Branch_Matrix[i][j] == 1:
            reward += 0.09
          else:
            reward += 0.01

    if self.counter == 50:                                                      # done-condition
      done = True

    info = {}

    return state, reward, done, info

  def render(self, mode):
    print(root)                                                                 # outputing current version of tree

    return

  def reset(self):
    root = build2(zero_tree)
    state = np.zeros(shape=(2,10,10), dtype=np.int)
    self.tree = zero_tree
    self.counter = 0

    return state

In [ ]:
env = Phylogenetic_Tree()
states = env.observation_space.shape
actions = tree_size*(seq_length+1)

**Test the enviroment**

In [ ]:
episodes = 0
for episode in range(1, 1+episodes):
  state = env.reset()
  score = 0
  done = False

  while not done:
    action = random.randrange(actions)                                          # creating random mutations, where '0' is non-mutation
    n_state, reward, done, info = env.step(action)
    score += reward
  env.render(mode='human')
  print('Episode:{} Score:{}'.format(episode, score))

**Create a Deep Learning Model with Keras**

In [ ]:
def build_model(states, actions):
  model = Sequential()
  model.add(Flatten(input_shape=((1, ) + states)))
  model.add(Dense(958, activation='sigmoid'))
  model.add(Dense(958, activation='sigmoid'))
  model.add(Dense(actions, activation='linear'))
  #model.add(Reshape((tree_size, seq_length+1), input_shape=(actions, )))
  return model

In [ ]:
model = build_model(states, actions)

**Build Agent with Keras-RL**

In [ ]:
def build_agent(model, actions):
  policy = BoltzmannQPolicy()
  memory = SequentialMemory(limit=50000, window_length=1)
  dqn = DQNAgent(model = model, memory = memory, policy = policy,
                 nb_actions = actions, nb_steps_warmup = 10, target_model_update = 1e-2)
  return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mean_squared_error'])
dqn.fit(env, nb_steps = 50000, visualize = False, verbose = 1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    4/10000 [..............................] - ETA: 3:54 - reward: -0.4950 

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    9/10000 [..............................] - ETA: 4:04 - reward: -0.2644

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 688s 69ms/step - reward: 1.8947
200 episodes - episode_reward: 94.733 [28.660, 119.770] - loss: 268.743 - mean_squared_error: 2358.852 - mean_q: 154.740

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 681s 68ms/step - reward: 1.8095
200 episodes - episode_reward: 90.476 [68.000, 129.000] - loss: 1093.049 - mean_squared_error: 9165.238 - mean_q: 345.514

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 681s 68ms/step - reward: 1.7986
200 episodes - episode_reward: 89.929 [65.000, 132.000] - loss: 1823.280 - mean_squared_error: 14467.529 - mean_q: 434.118

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 692s 69ms/step - reward: 1.7260
200 episodes - episode_reward: 86.301 [62.000, 134.000] - loss: 2404.666 - mean_squared_error: 19029.854 - mean_q: 495.281

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 693s

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 66.100, steps: 50
Episode 2: reward: 66.100, steps: 50
Episode 3: reward: 66.100, steps: 50
Episode 4: reward: 66.100, steps: 50
Episode 5: reward: 66.100, steps: 50
Episode 6: reward: 66.100, steps: 50
Episode 7: reward: 66.100, steps: 50
Episode 8: reward: 66.100, steps: 50
Episode 9: reward: 66.100, steps: 50
Episode 10: reward: 66.100, steps: 50
66.1


In [ ]:
dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...

0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0


0

Episode 1: reward: 66.100, steps: 50


In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)